In [ ]:
%%shell
pip3 install transformers>=4.32.0 optimum>=1.12.0
pip3 install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/  # Use cu117 if on CUDA 11.7

Looking in indexes: https://pypi.org/simple, https://huggingface.github.io/autogptq-index/whl/cu118/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 50.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 28.1 MB/s eta 0:00:00


In [ ]:
import json
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from IPython.display import FileLink
import torch

# Load WizardLM model and tokenizer
model_name_or_path = "TheBloke/WizardLM-1.0-Uncensored-Llama2-13B-GPTQ"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/837 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/7.26G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4371: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
The cos_cached attribute will be removed in 4.39. Bear in mind that its contents changed in v4.38. Use the forward method of RoPE from now on instead. It is not used in the `LlamaAttention` class
The sin_cached attribute will be removed in 4.39. Bear in mind that its contents changed in v4.38. Use the forward method of RoPE from now on instead. It is not used in the `LlamaAttention` class


generation_config.json:   0%|          | 0.00/162 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuratio

tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

In [ ]:
import csv
op_path = "/content/wizard_captions.csv"

def preprocess_captions(captions):
    # Define a set of prefixes to remove
    prefixes_to_remove = ["#C", "#O", "#B", "#X", "#Y"]

    cleaned_captions = []
    for caption in captions:
        # Check if the caption starts with any of the defined prefixes
        if any(caption.startswith(prefix) for prefix in prefixes_to_remove):
            # Remove the prefix and strip any leading/trailing whitespace
            cleaned_caption = caption
            for prefix in prefixes_to_remove:
                if cleaned_caption.startswith(prefix):
                    cleaned_caption = cleaned_caption[len(prefix):].strip()
                    break  # Stop after removing the first matching prefix
            cleaned_captions.append(cleaned_caption)

    return " ".join(cleaned_captions)




def preprocess_text(multi_para_text):
    # Replace newline characters with spaces to create a single line of text
    print(f" Multi para text : {multi_para_text}")
    return multi_para_text.replace('\n', ' ').strip()

def generate_summary(text):
    # Preprocess the input text to handle multiple paragraphs
    # processed_text = preprocess_text(text)

    # # Skip generating summary if the processed text is empty
    # if not processed_text:
    #     return None

    prompt = f'''Create a complete crisp summary, given a list of captions containing actions and potentially unidentified elements, generate a single, grammatically correct caption that combines the information coherently. Here are some guidelines: 1. Replace uppercase letters with 'person'. Eg: 'C walks' should be changed to 'a person walks'. Do not include references or notations in the final prompt at any cost 2. If the caption containins '#unsure', predict the missing information based on context from previous or the next captions. Eg: ['A person has a ball','He plays with '#unsure'']. Here, the person is most likely playing with the 'ball'. Hence, replace it with 'ball'.  But, if '#unsure' comes in the end of a caption, remove it from the caption and dont include in final prompt. 3. Combine the captions while maintaining chronological order and avoiding redundancy and ensure the final caption is grammatically correct and well-structured. 4. In case you notice 2 captions contradicting each other and are causing redundancy, consider the first one out of those 2 captions and ignore the second one from the final prompt. Example input: [ 'Person A scans surroundings, adjusts camera.', 'Person B shakes hands with #unsure in a room while looking around', 'Person walks around the room.' ]. Example output: A person scans their surroundings and adjusts a camera on their head. They then greet another person with a handshake while walking around the room. A third person walks around the room. Explanation: Explanation: 'another person' in caption 2 is likely the person from caption 1 based on context. So, we replaced '#unsure' with 'another person'. I repeat, pay a lot of attention to all the guidelines mentioned above. Your final prompt MUST NOT contain any upper-case alphabet notations or '#unsure'.Here is the list of captions. Rely strictly on the provided text, without including external information. Do not stop the sentence abruptly, for example, don't end like "C drops contai" or "C drops, finally". It must be a completed sentence. If the given text to summarize is small, then don't purposely make it lengthy. Do not cut off the sentence in the middle and it has to end with a full stop:
{text}'''

    prompt_template = f'''Create a complete crisp summary, given a list of captions containing actions and potentially unidentified elements, generate a single, grammatically correct caption that combines the information coherently. Here are some guidelines: 1. Replace uppercase letters with 'person'. Eg: 'C walks' should be changed to 'a person walks'. Do not include references or notations in the final prompt at any cost 2. If the caption containins '#unsure', predict the missing information based on context from previous or the next captions. Eg: ['A person has a ball','He plays with '#unsure'']. Here, the person is most likely playing with the 'ball'. Hence, replace it with 'ball'.  But, if '#unsure' comes in the end of a caption, remove it from the caption and dont include in final prompt. 3. Combine the captions while maintaining chronological order and avoiding redundancy and ensure the final caption is grammatically correct and well-structured. 4. In case you notice 2 captions contradicting each other and are causing redundancy, consider the first one out of those 2 captions and ignore the second one from the final prompt. Example input: [ 'Person A scans surroundings, adjusts camera.', 'Person B shakes hands with #unsure in a room while looking around', 'Person walks around the room.' ]. Example output: A person scans their surroundings and adjusts a camera on their head. They then greet another person with a handshake while walking around the room. A third person walks around the room. Explanation: Explanation: 'another person' in caption 2 is likely the person from caption 1 based on context. So, we replaced '#unsure' with 'another person'. I repeat, pay a lot of attention to all the guidelines mentioned above. Your final prompt MUST NOT contain any upper-case alphabet notations or '#unsure'.Here is the list of captions. Rely strictly on the provided text, without including external information. Do not stop the sentence abruptly,for example, don't end like "C drops contai" or "C drops, finally".It must be a completed sentence. If the given text to summarize is small, then don't purposely make it lengthy. Do not cut off the sentence in the middle and it has to end with a full stop:

USER: {prompt}
ASSISTANT:
'''

    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=512,  # Adjust based on model capability
        do_sample=True,
        temperature=0.2,
        eos_token_id=tokenizer.eos_token_id
    )

    response = pipe(prompt_template)[0]['generated_text']
    generated_summary = response[len(prompt_template):].strip()

    return generated_summary if generated_summary else None  # Return None if summary is empty

def write_csv(x,y):
   with open('/content/wizard_captions.csv', 'a', encoding="utf-8", newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    for a,b in zip(x,y):
      csv_writer.writerow([a,b])



def generate_video_summaries(input_file_path):
    with open(input_file_path, 'r') as file:
        data = json.load(file)

    # updated_json = {}
    keys = list(data.keys())
    ref_captions=[]

    for key in keys:
      ref_captions.append(data[key])


    for i in range(0,len(keys),300):
      start=i
      end=min(i+300,len(keys))
      key_temp = keys[start:end]
      ref_temp = ref_captions[start:end]
      gen_temp=[]

    # for video_id, captions in data.items():
        # Preprocess captions for the video ID
        # processed_captions = preprocess_captions(captions)

        # Generate summary for the processed captions
      gen_temp = [generate_summary(caption) for caption in ref_temp] # Adjust max_words as needed
      # print(f"Summary: {summary}")
      print(f"Writing from {start} to {end}!!")
      write_csv(key_temp,gen_temp)
      # Store the summarized caption for the video ID
      # updated_json[video_id] = summary

    # Save the updated content to a new JSON file
    # with open(output_file_path, 'w') as file:
    #     json.dump(updated_json, file, indent=4)

    # return output_file_path

def generate_video_summary():
  captions ="C picks two mac and cheese boxes from a carton with both hands. C opens a plastic bucket lid with her right hand. C places the mac and cheese boxes in the plastic bucket with her left hand."
  summary = generate_summary(captions)
  print(summary)


# Input file path and output file path
input_file_path = '/content/reference_captions_final.json'
# output_file_path = 'summarized_captions.json'

# Generate summaries for the captions and save to output file
# generate_video_summaries(input_file_path, output_file_path)
generate_video_summaries(input_file_path)

# Provide a link to download the summarized output JSON file
# FileLink(output_file_path)



In [ ]:
import locale

locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
# !pip install rouge

le = [1,2,3]

re = [(x+1) for x in le]
print(re)

[2, 3, 4]
